# PostgreSQL Integration Test Guide

This guide will help you with running Integration Tests for PostgreSQL with JupySQL

To run this tutorial you will need to start with setting up a local development environment for JupySQL. 

### Step 1 
To setup a local development environment follow setup guide over here :
[Setup Guide](https://ploomber-contributing.readthedocs.io/en/latest/contributing/setup.html)

### Step 2
Once the local environment is setup for JupySQL, activate the environment by running

```sh
conda activate jupysql
```

### Step 3

For testing PostgreSQL Integration we will need to install following dependencies 

### Install PostgreSQL client

To connect to a PostgreSQL database from Python, you need a client library. We recommend using `psycopg2`, but there are others like `pg8000`, and `asyncpg`. JupySQL supports the [following connectors.](https://docs.sqlalchemy.org/en/14/dialects/postgresql.html#dialect-postgresql)

#### Installing `psycopg2`
If you have `conda` installed, it is more reliable to use it:

```sh
conda install psycopg2 -c conda-forge
```

#### Installing `pgspecial`

Ensure that you are using `pgspecial 1.x`. `pgspecial 2.x` has migrated to `psycopg3` and thus does not yield informative error messages.

```sh
conda install "pgspecial<2" -c conda-forge
```

#### Install dockerctx

dockerctx is a context manager for managing the lifetime of a docker container. We'll be using this library as we're spinning up docker container to launch the database instance and running integration tests over it.


```{tip}
Make sure the earlier dependencies - psycopg2, pgspecial are installed before installing dockerctx, otherwise it might throw an error
```


In [ ]:
%pip install dockerctx

If you have trouble getting it to work, [message us on Slack.](https://ploomber.io/community)

Now as we have the required dependencies, we will spin up a PostgreSQL docker instance to test our integration with the database.

To get PostgreSQL instance up and running, you need to install following Python packages:

In [ ]:
%pip install jupysql pandas pyarrow --quiet

Note: you may need to restart the kernel to use updated packages.


You also need a PostgreSQL connector. Here's a list of [supported connectors.](https://docs.sqlalchemy.org/en/14/dialects/postgresql.html#dialect-postgresql) We recommend using `psycopg2`. The easiest way to install it is via:

```{tip}
If you have issues, check out our [installation guide](../howto/postgres-install.md) or [message us on Slack.](https://ploomber.io/community)
```

You also need Docker installed and running to start the PostgreSQL instance.

## Start PostgreSQL instance

We fetch the official image, create a new database, and user (this will take 1-2 minutes). We are using credentials as defined in test config file at ```"src/sql/_testing.py"```

In [ ]:
%%bash
docker run --name postgres -e POSTGRES_DB=db \
  -e POSTGRES_USER=ploomber_app \
  -e POSTGRES_PASSWORD=ploomber_app_password \
  -p 5432:5432 -d postgres

fe9f1cf0d371152c64bdedfaa56c13e655a1352068b4c2b616cdf8bb5c3327a8


```{important}
Check if docker container is up and running before proceeding forward
```

### Running Integration Tests

Now, as we have PostgreSQL docker instance set up we can go ahead and run the integration tests. Integrations tests are located at ```"src/tests/integration/"``` we will run them using following commands.

```sh
pytest src/tests/integration/test_postgreSQL.py
```

If everything goes well, you'll see the tests Passed as the output after running the command

The other set of tests which test for generic db operations are located in ```"src/tests/integration/test_generic_db_operations.py"```. We will run them for our PostgreSQL DB by running following command

```sh
pytest src/tests/integration/test_generic_db_operations.py -k "ip_with_postgreSQL"
```

```"-k"``` flag in above command helps us to filter and run the tests which are only specific to PostgreSQL DB integration

### Common Errors

```{Important}
The following error might show up if docker is not installed and/or the container is not running. Hence make sure to check if PostgrSQL docker instance is up and running before executing the pytest commands
```

```docker.errors.DockerException: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))```

## Clean up

To stop and remove the container:

In [ ]:
! docker container ls

CONTAINER ID   IMAGE      COMMAND                  CREATED         STATUS         PORTS                    NAMES
fe9f1cf0d371   postgres   "docker-entrypoint.s…"   6 seconds ago   Up 5 seconds   0.0.0.0:5432->5432/tcp   postgres


In [ ]:
%%capture out
! docker container ls --filter ancestor=postgres --quiet

In [ ]:
container_id = out.stdout.strip()
print(f"Container id: {container_id}")

Container id: 


In [ ]:
! docker container stop {container_id}

"docker container stop" requires at least 1 argument.
See 'docker container stop --help'.

Usage:  docker container stop [OPTIONS] CONTAINER [CONTAINER...]

Stop one or more running containers


In [ ]:
! docker container rm {container_id}

"docker container rm" requires at least 1 argument.
See 'docker container rm --help'.

Usage:  docker container rm [OPTIONS] CONTAINER [CONTAINER...]

Remove one or more containers


In [ ]:
! docker container ls

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES
